# Logging DLPy Scalar Statistics in TensorBoard
This notebook shows how to use the TensorBoard API in DLPy to track scalar statistics from the model training process. Using [TensorBoard](https://www.tensorflow.org/tensorboard) can help organize experiments, provide real-time updates on training progression, graph visualization, and more. You can use TensorBoard in the browser as well as inside a Jupyter Notebook. Here we show how to use the tensorboard magic to surface the application inside the notebook.

To run this example, make sure to install TensorFlow 2.0 and Jupyter. Preferably use a virtual environment with a fresh install of all the needed dependencies. Follow instructions [here](https://www.tensorflow.org/install/pip) for TensorFlow installation. See [here](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks) for more information on TensorBoard integration with Jupyter.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import tensorflow as tf
print('TensorFlow version - ', tf.__version__)

import swat
import dlpy
from dlpy import Sequential
from dlpy.splitting import two_way_split
from dlpy.images import ImageTable
from dlpy.tensorboard import TensorBoard
from dlpy.model import *
from dlpy.layers import * 
from dlpy.applications import *
from dlpy.tensorboard import TensorBoard

DLPY_DATA_DIR = './python-dlpy/dlpy/tests/datasources/'

## Connect to CAS and Load Data

In [ ]:
s = swat.CAS('host_name', port_number)
images = ImageTable.load_files(s, path=DLPY_DATA_DIR + 'giraffe_dolphin_small')
train, test = two_way_split(images)
train.resize(224, 224, inplace=True)
test.resize(224, 224, inplace=True)

In [ ]:
train.label_freq

In [ ]:
test.label_freq

## Build Model

In [ ]:
model1 = Sequential(s, model_table='Simple_CNN1')
model1.add(InputLayer(3, 224, 224))
model1.add(Conv2d(8, 7))
model1.add(Pooling(2))
model1.add(Conv2d(8, 7))
model1.add(Pooling(2))
model1.add(Dense(16))
model1.add(OutputLayer(act='softmax', n=2))

## Launch TensorBoard
The following cell uses the `%tensorboard` magic to launch the TensorBoard application inside the output window of the executed cell. At minimum you must pass the `logdir` parameter to `tensorboard`. This provides a directory to write log files to and tells the application where to read the corresponding files from. Below we set our logdir to the `./data` directory.

If you prefer to use TensorBoard as a standalone application, outside of Jupyter, you can run the below command on the command line and access the web application on the assigned port (usually 6006). 

In [ ]:
%tensorboard --logdir './data/'

## Instantiate TensorBoard Object
The TensorBoard API is fairly simple as it just expects the model you would like to monitor, the logdir directory used to start TensorBoard, and optionally logging validation statistics. By default only `learning_rate`, `fit_error`, and `loss` are recorded.

In [ ]:
LOG_DIR = './data/'
tensorboard = TensorBoard(model1, LOG_DIR, use_valid=True)

## Train Model and View TensorBoard
While the model trains you can track its progress in the TensorBoard window above in real-time. Each scalar metric contains its own graph. So, anytime you train a new model or want to change the training parameters for a model you can easily compare these different models as they will be shown on the same graph. Here we use the fit_tensorboard() method which provides the same functionality as fit() but includes tensorboard logging. 

In [ ]:
solver = MomentumSolver(learning_rate=0.00001, clip_grad_max = 100, clip_grad_min = -100)
optimizer = Optimizer(algorithm=solver, mini_batch_size=8, log_level=2, max_epochs=100, reg_l2=0.0005)
model1.fit_tensorboard(data=train, valid_table=test, optimizer=optimizer, tensorboard=tensorboard)

## Experiments
Here we train two new model while also training our initial model for a few more epochs. One of the new models shares the same architecture as our original model but contains different training hyperparameter values while the other is an entirely different architecture. The idea is to show how you can more easily keep track of your experiments. You can filter out different models from within the TensorBoard window to organize the visualizations.

In [ ]:
solver = MomentumSolver(learning_rate=0.00001, clip_grad_max = 100, clip_grad_min = -100)
optimizer = Optimizer(algorithm=solver, mini_batch_size=8, log_level=2, max_epochs=20, reg_l2=0.0005)
model1.fit_tensorboard(data=train, valid_table=test, optimizer=optimizer, tensorboard=tensorboard)

In [ ]:
model2 = Sequential(s, model_table='Simple_CNN2')
model2.add(InputLayer(3, 224, 224))
model2.add(Conv2d(8, 7))
model2.add(Pooling(2))
model2.add(Conv2d(8, 7))
model2.add(Pooling(2))
model2.add(Dense(16))
model2.add(OutputLayer(act='softmax', n=2))

In [ ]:
tensorboard2 = TensorBoard(model2, LOG_DIR, use_valid=True)
solver = MomentumSolver(learning_rate=0.003, clip_grad_max = 100, clip_grad_min = -100)
optimizer = Optimizer(algorithm=solver, mini_batch_size=8, log_level=2, max_epochs=100, reg_l2=0.0005)
model2.fit_tensorboard(data=train, valid_table=test, optimizer=optimizer, tensorboard=tensorboard2)

In [ ]:
from dlpy.applications import InceptionV3

model3 = InceptionV3(s, model_table='InceptionV3', n_classes=2, n_channels=3, width=224, height=224)

In [ ]:
tensorboard3 = TensorBoard(model3, LOG_DIR, use_valid=True)
solver = MomentumSolver(learning_rate=0.003, clip_grad_max = 100, clip_grad_min = -100)
optimizer = Optimizer(algorithm=solver, mini_batch_size=8, log_level=2, max_epochs=25, reg_l2=0.0005)
model3.fit_tensorboard(data=train, valid_table=test, optimizer=optimizer, tensorboard=tensorboard3)